need to write python minibatch generator from file

In [2]:
import pandas as pd
import os
import glob
from scipy import misc
import numpy as np

In [3]:
y = pd.read_csv(os.path.join('..', 'data', 'trainLabels.csv'))

In [4]:
import itertools

In [5]:
class ImageGenerator:
    def __init__(self, batch_size=5, X_path=os.path.join("E:", "DR_Data", "Train_512"), y_path=os.path.join('..', 'data', 'trainLabels.csv'), shape=(3, 512, 512), ending=".jpeg"):
        self.X_path = X_path
        self.y_path = y_path
        self.shape = shape
        self.ending = ending
        self.batch_size = batch_size

    def reshape(self, im, shape):
        im = misc.imread(im)
        return np.reshape(im, shape)

    def x_generator(self, directory):
        for image in glob.glob(os.path.join(directory, '*'+self.ending)):
            yield self.reshape(image, self.shape)
            
    def y(self, directory):
        y = pd.read_csv(directory)
        return y.level.values

    def __iter__(self):
        x_gen = self.x_generator(self.X_path)
        y = self.y(self.y_path)
        start = 0
        total = self.batch_size
        while total < len(y):
            X = np.array(list(itertools.islice(x_gen, start, start+self.batch_size, 1))), np.array(list(itertools.islice(y, start, start+self.batch_size, 1)))
            y = np.array(list(itertools.islice(y, start, start+self.batch_size, 1)))
            yield X, y


In [6]:
ig = ImageGenerator()

In [7]:
for i, y in ig:
    print i.shape, y.shape

AttributeError: 'tuple' object has no attribute 'shape'

In [54]:
y = pd.read_csv(os.path.join('..', 'data', 'trainLabels.csv'))

In [78]:
y_new=y.sample(frac=0.8,random_state=1337)
test=y.drop(y_new.index)
train= y_new.sample(frac=0.8,random_state=1337)
val = y_new.drop(train.index)

In [79]:
train.shape

(22481, 2)

In [80]:
test.shape

(7025, 2)

In [82]:
train.head()

,image,level
883,1058_right,0
29279,37069_right,0
13898,17445_left,0
6533,8221_right,0
34904,44093_left,0


In [83]:
val.head()

,image,level
14162,17785_left,1
6315,7944_right,0
402,458_left,3
8501,10660_right,0
22525,28409_right,2


In [84]:
test.head()

,image,level
0,10_left,0
3,13_right,0
6,16_left,4
10,19_left,0
14,21_left,0


In [90]:
import shutil

In [86]:
def find_class(df, label):
    return df[df.level == label].image.values

In [128]:
def move_file(name, split, label, src=os.path.join("E:", "DR_Data"), dst=os.path.join("E:", "DR_Data"), ending='.jpeg'):
    shutil.move(os.path.join(src, split+'_512', name+ending), os.path.join(dst, split, str(label), name+ending))

In [129]:
def split_classes(df, split, labels=[0, 1, 2, 3, 4], src=os.path.join("E:", "DR_Data"), dst=os.path.join("E:", "DR_Data"), ending='.jpeg'):
    for label in labels:
        names = find_class(df, label)
        for name in names:
            move_file(name, split, label, src, dst, ending)
    

In [130]:
zeros_train = find_class(train, 0)

In [133]:
move_file(zeros_train[0], 'train', '0')

IOError: [Errno 2] No such file or directory: 'E:DR_Data\\train_512\\1058_right.jpeg'

In [135]:
split_classes(train, 'train')